In [1]:
# docker pull emacski/tensorflow-serving:latest-linux_arm64
# docker run -t --rm -p 8501:8501 --mount type=bind,source=/tmp/model_name/,target=/models/model_name/ -e MODEL_NAME=model_name emacski/tensorflow-serving:latest-linux_arm64

In [25]:
#!docker start #como iniciar da  linha de comando
!open --background -a Docker

In [27]:
!docker run hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (arm64v8)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



In [33]:
#!docker pull tensorflow/serving

In [95]:
# esse funciona para quem n tem m1
#!docker pull tensorflow/serving:latest-gpu

In [22]:
# conda install tensorflow

In [34]:
import tensorflow as tf

In [43]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input, decode_predictions
import numpy as np

In [7]:
model = EfficientNetV2B0(weights='imagenet')

29403144/29403144 [==============================] - 40s 1us/step


In [8]:
pwd

'/Users/csamp/Documents/stack_ab'

In [76]:
img_path = 'tf_serving/lion.jpg'

img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)

print('Predicted:', decode_predictions(preds, top=3)[0])

1/1 [==============================] - 0s 77ms/step
Predicted: [('n06596364', 'comic_book', 0.17904864), ('n03590841', "jack-o'-lantern", 0.066323765), ('n03291819', 'envelope', 0.039136156)]


In [77]:
import time
current_time = int(time.time())
path = f"effv2b0/{current_time}"
model.save(path)

INFO:tensorflow:Assets written to: effv2b0/1670445686/assets


INFO:tensorflow:Assets written to: effv2b0/1670445686/assets


In [78]:
path

'effv2b0/1670445686'

In [3]:
# # how the folder should look like
# ├── effv2b0
# │ ├── 1670405165
# │ │ ├── assets
# │ │ ├── saved_model.pb
# │ │ └── variables

In [26]:
# esse funciona
#!docker pull emacski/tensorflow-serving:latest-linux_arm64

latest-linux_arm64: Pulling from emacski/tensorflow-serving
Digest: sha256:37ae953d10a10f520fc7aca068c866f28a424894c47b1d06c8cec22e631e29be
Status: Image is up to date for emacski/tensorflow-serving:latest-linux_arm64
docker.io/emacski/tensorflow-serving:latest-linux_arm64


In [94]:
# !docker run -t --rm -p 8501:8501 \
#     -v "/Users/csamp/Documents/stack_ab/effv2b0/:/models/effv2b0/" \
#     -e MODEL_NAME=effv2 \
#     emacski/tensorflow-serving:latest-linux_arm6

In [50]:
#!docker run -p 8501:8501 --name tfserving_vgg16 --mount type=bind,source=vgg16/,target=vgg16/ -e MODEL_NAME=vgg16 -t tensorflow/serving

In [87]:
#!docker run -p 8502:8502 --name ttfserving_vgg -v source=/Users/csamp/Documents/stack_ab/vgg16/,target=vgg16 -e MODEL_NAME=vgg16 -t emacski/tensorflow-serving:latest-linux_arm64

In [13]:
pwd

'/Users/csamp/Documents/stack_ab'

In [35]:
#docker pull tensorflow/serving
# esse funciona
!docker run --rm -p 8501:8501 \
        --name tfserving_effv2 \
        -v "/Users/csamp/Documents/stack_ab/effv2b0:/models/effv2b0" \
        -e MODEL_NAME=effv2b0 \
        emacski/tensorflow-serving:latest-linux_arm64

2022-12-07 20:31:25.278289: I external/tf_serving/tensorflow_serving/model_servers/server.cc:89] Building single TensorFlow model file config:  model_name: effv2b0 model_base_path: /models/effv2b0
2022-12-07 20:31:25.278676: I external/tf_serving/tensorflow_serving/model_servers/server_core.cc:465] Adding/updating models.
2022-12-07 20:31:25.278691: I external/tf_serving/tensorflow_serving/model_servers/server_core.cc:591]  (Re-)adding model: effv2b0
2022-12-07 20:31:25.394043: I external/tf_serving/tensorflow_serving/core/basic_manager.cc:740] Successfully reserved resources to load servable {name: effv2b0 version: 1670405165}
2022-12-07 20:31:25.394086: I external/tf_serving/tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: effv2b0 version: 1670405165}
2022-12-07 20:31:25.394118: I external/tf_serving/tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: effv2b0 version: 1670405165}
2022-12-07 20:31:25.394712: I external/

In [109]:
# API endpoint can be accessed at http://localhost:8501. 

In [72]:
import json
import requests

def run_prediction(data,headers,endpoint):
    json_response = requests.post(endpoint,data=data,headers=headers)
    prediction = json.loads(json_response.text)
    return (decode_predictions(np.array(preds), top=3)[0])

In [73]:
json.loads(json_response.text)

{'error': 'Servable not found for request: Latest(effv2)'}

In [79]:
endpoint = 'http://localhost:8501/v1/models/effv2:predict'
data = json.dumps({"instances": x.tolist()})
headers = {"content-type": "application/json"}
run_prediction(data,headers,endpoint)

[('n06596364', 'comic_book', 0.17904864),
 ('n03590841', "jack-o'-lantern", 0.066323765),
 ('n03291819', 'envelope', 0.039136156)]

In [103]:
# code from https://cnvrg.io/how-to-serve-a-model-with-tensorflow/
# another way: https://medium.com/analytics-vidhya/deploy-production-ready-keras-model-with-tensorflow-serving-and-docker-85615245a9ed

In [75]:
json_response = requests.post('http://localhost:8501/v1/models/effv2:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n06596364', 'comic_book', 0.17904864), ('n03590841', "jack-o'-lantern", 0.066323765), ('n03291819', 'envelope', 0.039136156)]


In [59]:
# ran docker with
# docker run -t --rm -p 8501:8501 \
#     -v "/Users/csamp/Documents/stack_ab/vgg16/:/models/vgg16/" \
#     -e MODEL_NAME=vgg16 \
#     emacski/tensorflow-serving:latest-linux_arm64

I ran docker with:

`
docker run -t --rm -p 8501:8501 \
    -v "/Users/csamp/Documents/stack_ab/vgg16/:/models/vgg16/" \
    -e MODEL_NAME=vgg16 \
    emacski/tensorflow-serving:latest-linux_arm6`
`

docker run -t --rm -p 8501:8501 \ 
    --mount type=bind,source=/tmp/model_name/,target=/models/model_name/ 
    -e MODEL_NAME=model_name \ 
    emacski/tensorflow-serving:latest-linux_arm64

In [17]:
!docker run -t --rm -p 8501:8501 \
    -v "/Users/csamp/Documents/stack_ab/effv2/:/models/effv2/" \
    -e MODEL_NAME=vgg16 \
    emacski/tensorflow-serving:latest-linux_arm6

docker: Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.
